# SDSS eBOSS Data 
## Script on reading and pre-processing data, and generation of a catalogue of desirable galaxy types

This script extracts useful data from the spPlate and spAll_redrock fits files, and generates the required training data set.

1. **Defining input parameters**
2. **Reading and pre-processing the data**
3. **Applying selection cuts** <br>
   3.1 **Selecting common wavelengths**
5. **Generating the training data set**

**Data**: 14th Oct, 2019. <br>
**Author**: Soumya Shreeram <br>
**Supervised by**: Anand Raichoor <br>
**Script adapted from**: S. Ben Nejma


In [1]:
import astropy.io.fits as fits
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import numpy as np
from numpy.lib.format import open_memmap
import os
import subprocess
from astropy.convolution import convolve, Box1DKernel

## 1. Defining input parameters

In [2]:
# data directory on lesta with the spAll_redrock files
spPlate_dir = r'/hpcstorage/raichoor/spplatelist_v5_13_0/spPlate'
spAll_redrock_file = r'/hpcstorage/raichoor/spplatelist_v5_13_0/' \
            'spall_redrock_v5_13_0.valid.fits'

# number of plates of ELGs, LRG+QSOs, and Boss plates to select
num_pl = 4 

# return info about selected fibres?
return_infos = True

# select corrupted or uncorrunpter samples?
uncorrupted = False

# minimum - maximum wavelengths considered
wmin, wmax = 3700, 10000

## 2. Reading the data

In [3]:
def setName(data_dir, plate_mjd):
    file_name = data_dir+'-'+plate_mjd+'.fits'
    return file_name

def readFile(filename):
    """
    Function opens the file
    @input filename :: name of the file
    """
    hdu = fits.open(filename)
    data = hdu[1].data
    hdu.close()        
    return data

def plateMJD(data):
    # defining the PLATE number, p, and MJD, m for all the files
    pms = np.array([str(p)+'-'+ str(m) for p, m in zip(data['PLATE'],
data['MJD'])])
    return pms

def uniquePmsProgramme(pms, data):
    # selecting only the unique plates-mjd, and find their programmes
    pms_unique, idx = np.unique(pms, return_index=True)
    prog_unique = data['programname'][idx]
    return pms_unique, prog_unique

def applyWaveCuts(w, wmin, wmax):
    idx = np.where((w>wmin) & (w<wmax))[0]
    return w[idx], idx      

def readSpPlate(data_dir, plate_mjd, wmin, wmax):
    """
    Function to read the useful headers and data from spPlate fits file
    @param place :: 4-digit plate number
    @param mjd :: 5-digit MJD
    
    @returns wavelength, bunit, flux, ivar (refer comments for individual meanings)
    """
    # opens the file
    hdu     = fits.open(setName(data_dir, plate_mjd))        
    
    c0      = hdu[0].header['coeff0']   # Central wavelength (log10) of first pixel
    c1      = hdu[0].header['coeff1']   # Log10 dispersion per pixel
    npix    = hdu[0].header['naxis1']   # WIDTH (TOTAL!
    wavelength    = 10.**(c0 + c1 * np.arange(npix))
    bunit   = hdu[0].header['bunit']    # Units of flux

    flux    = hdu[0].data               # Flux in units of 10^-17^ erg/s/cm^2^/Ang
    ivar    = hdu[1].data               # Inverse variance (1/sigma^2^) for HDU 0
    hdu.close()
    
    # do cuts on wavelength
    if wmin != None:
        wavelength, widx = applyWaveCuts(wavelength, wmin, wmax)
        
    return wavelength, widx, bunit, flux, ivar

def writeToFile(pms, outfilename, selected_plates, username):
    """
    Function extracts the info from desired files and writes to a new file
    @param pms :: complete array of plate nos. and MJD
    @param outfilename :: output file name
    @param selected_plates :: array of all the selected plates
    """    
    # extract those plate-mjd files
    extract_files = np.in1d(pms, selected_plates)
    output = outfilename+username
    
    # write info to new fits file
    hdu[1].data = hdu[1].data[extract_files]
    return hdu.writeto(outfilename, overwrite=True)

def writeOutputToFile(input_name, shape_arr, in_dtype):
    """
    Write to a .npy file as a memory-mapped array
    @param input_name :: array name
    @param shape_arr :: shape of the array to be memory-mapped
    
    @return output_arr :: the memory-mapped array
    """
    filename = 'Data_files/'+input_name+'.npy'
    w1 = open_memmap(filename, dtype=in_dtype, mode='w+', shape=shape_arr)
    return w1

In [4]:
# reads the file spAll_redrock and generates arrays of unique plate-MJD and programs
data = readFile(spAll_redrock_file)
pms = plateMJD(data)

## 3. Applying selection cuts

The functions below implement various selection cuts to obtain the desired data. They are summarized below:
* Select plates that observe **E**mission-**L**ine type **G**alaxies (ELGs), LRGs, and QSOs
* Select wavelength that are common to all plates
* Removing sky spectra and certain configurations
* Select redshift range (Zspec fibres)

In [5]:
def galaxyType(pms_unique, prog_unique, names, gal_type, num_p):
    """
    Function chooses the file name based of desired galaxy type
    @params pms_unique, prog_unique :: unique array of plate nos.-MJD & programmes
    @param names :: array of names of the galaxies/programmes to select/de-select
    @param gal_type :: string to distinguish the desired operations
    @param num_p :: number of plates of each galaxy to select
    
    @returns sub_plates :: bool indicies of selected plates within pms unique
    """
    if gal_type == 'ELG': # select ELG plates
        sub_plates = np.random.choice(pms_unique[(prog_unique==names[0]) | \
                                    (prog_unique==names[1])],size=num_p).tolist()
    elif gal_type == 'LRG+QSO': # select LRG+QSO plates
         sub_plates = np.random.choice(pms_unique[(prog_unique==names[0]) & \
                                            (prog_unique!=names[1]) & \
                                            (prog_unique!=names[2])],size=num_p).tolist()
    else: # select boss plates
        sub_plates = np.random.choice(pms_unique[(prog_unique==names[0])],size=num_p).tolist()    
    return np.in1d(pms_unique, sub_plates)

    
def selectPlates(pms_unique, prog_unique, num_pl):
    """
    Function the selects plates containing ELGs, LRG+QSOs, and some random.
    @param pms_unique :: arroy of plate nos. and MJDs
    @param prog_unique :: list of unique programmes (eBoss/Boss)
    @param num_pl :: number of plates of each category to select
    
    @returns selected_plates :: array of the file names containing desired galaxies
    """
    selected_plates = np.zeros(len(pms_unique), dtype = bool)
    
    # select <<num_pl>> eboss ELG plates
    names_elg = ['ELG_NGC', 'ELG_SGC']
    selected_plates |= galaxyType(pms_unique, prog_unique, names_elg, 'ELG', num_pl)
    
    # select <<num_pl>> eboss LRG+QSO plates
    names_lrgQso = ['eboss', 'ELG_NGC', 'ELG_SGC']
    selected_plates |= galaxyType(pms_unique, prog_unique, names_lrgQso, 'LRG+QSO', num_pl)
    
    # select <<num_pl>> random boss plates
    names_boss = ['boss']
    selected_plates |= galaxyType(pms_unique, prog_unique, names_boss, 'boss plates', num_pl)
    
    return selected_plates

def discardSkySpectra(data):
    """
    Function derfines the conditions for discarding the sky spectra
    and some othe factors
    """
    data = data[(data['ZWARN'] == 0) & \
                (data['OBJTYPE'] != 'SKY') & \
                (data['CHI2']/data['NPIXELS'].astype(float) > 0.4) & \
                (data['DELTACHI2']/data['NPIXELS'].astype(float) > 0.0025) &\
                (data['OBJTYPE'] != 'SPECTROPHOTO_STD')] # deletes stars - flux cal.  
    return data

def selectCommonWaves(pms_unique, spPlate_dir, wmin, wmax):
    """
    Function keeps common wavelengths observed in all files in range [wmin, wmax]
    
    """
    npix, delta, wave_indicies = [], [], []
    for i, pm in enumerate(pms_unique):
        if i == 0:
            # wavelength saved from 1st file
            w_i, idx_i = readSpPlate(spPlate_dir, pm, wmin, wmax)[0:2]        
            npix.append(len(w_i))
            delta.append(0)
            wave_indicies.append(idx_i)
        else:
            w, idx = readSpPlate(spPlate_dir, pm, wmin, wmax)[0:2]        
            npix.append(len(w))
            delta.append(np.unique(np.abs(w-w_i)).max())
            # keeping tract of all the opened wavelength indicies
            wave_indicies.append(idx[np.where(idx != idx_i)])
    return npix, delta, w, wave_indicies

In [6]:
# sky spectra and other factors taken out
data_i = discardSkySpectra(data)
pms_i = plateMJD(data_i)

# find unique pms, programmes
pms_unique, prog_unique = uniquePmsProgramme(pms_i, data_i)

# select plates containing ELGs, LRGs, QSOs, and some boss plates
keep = selectPlates(pms_unique, prog_unique, num_pl)
pms_unique, prog_unique = pms_unique[keep], prog_unique[keep]

# print the selected plates
print("Plate-MJD  | Programme")
for pm, prog in zip(pms_unique, prog_unique):     
    print(pm, " | ", prog)

Plate-MJD  | Programme
11109-58523  |  eboss
11342-58425  |  eboss
3686-55268  |  boss
5150-56329  |  boss
5383-56013  |  boss
6012-56101  |  boss
7677-57363  |  eboss
8524-57656  |  eboss
9372-58074  |  ELG_SGC
9442-58076  |  ELG_SGC
9611-58136  |  ELG_NGC


#### 3.1  Saves common wavelengths

In [7]:
# applies cuts on wavelengths, and stores all the indicies of selected wavelengths
npix, delta, wave, wave_indicies = selectCommonWaves(pms_unique, spPlate_dir, wmin, wmax)

# print some info        
print("Total no. of unique wavelengths, ω = %.1f.\
      \nMax. difference in wavelegths between the %d files Δω = %1.2e"%(np.unique(npix),len(pms_unique),np.max(delta)))
print("\nShape of wavelength indicies for %d files:"%len(pms_unique), np.shape(wave_indicies))

# saves the wavelength in a memory-mapped array
w = writeOutputToFile('wavelength', np.shape(wave), 'float32')
w[:] = wave
del w

Total no. of unique wavelengths, ω = 4317.0.      
Max. difference in wavelegths between the 11 files Δω = 1.09e-11

Shape of wavelength indicies for 11 files: (11,)


### 4. Generating the training data sets

In [26]:
def listOfFilenamesIDs(data, pms, return_infos, pms_unique):
    """
    Function generates list of filenames and index IDs for selected fibres
    @param data :: data from the spAllRedrock fiel
    @param pms :: plate no. MJD info array of the spPlate files
    """
    nb_fibres = 0
    for i in range(len(pms_unique)):
        nb_fibres += len(selectd_fibres[i][:])
    shape_info = (nb_fibres,)
    
    # list of fibre ids for selected files
    all_selected_idx = np.array([np.array(data[pms == pl_mjd]['FIBERID']-1) for pl_mjd in pms_unique])
    
    if return_infos:
        info = writeOutputToFile('info', shape_info, '<U30')
        # [TODO] need to sort this out  
        #info[:,0] = pms_i
        # their fibre ids
        info[:] = np.array([str(f_id) for selected_ids in all_selected_idx for f_id in selected_ids])
        
        del info
    return all_selected_idx, nb_fibers

def fileNameMaps(uncorrupted, nb_fibers, wave_idxs):
    if uncorrupted:
        x_name = writeOutputToFile('X_corrupted', (nb_fibers, len(wave_idxs)), 'float32')
        y_name = writeOutputToFile('Y_corrupted', (nb_fibers,), 'float32')
    else:
        x_name = writeOutputToFile('X', (nb_fibers, len(wave_idxs)), 'float32')
        y_name = writeOutputToFile('Y', (nb_fibers,), 'float32')    
    counter = 0
    return counter, x_name, y_name

In [67]:
# prints list of selected fibres
return_infos=True
selectd_fibres , nb_fibers = listOfFilenamesIDs(data_i, pms_i, return_infos, pms_unique)

# sets name according to nature of samples for: classification vs detection
counter, x_name, y_name = fileNameMaps(uncorrupted, nb_fibers, wave_indicies[2][:])

In [ ]:
for i, (pm, fib_ids) in enumerate(pms_unique, selectd_fibres):
    # gets the flux
    flux = readSpPlate(data_dir, plate_mjd, wmin, wmax)[3]
    
    X[counter:counter+len(fib_ids)][:] = np.array(flux[:, ])
    

In [72]:
print(np.shape(x_name))
print(len(wave_indicies[2][:]))

(11, 4317)
4317
